# Preliminary API

In [1]:
import TWB
import glob

## Load data

In [2]:
# Make sure that the tags are also wikipedia articles!!!

tags = [ 'humanitarian crisis',
         'natural disaster',
         'environmental crisis',
         'disability',
         'gender',
         'genital mutilation',
         'racism',
         'genocide',
         'civil war',
         'terrorism',
         'infectious disease',
         'political revolution',
         'political prisoner',
         'amnesty',
         'corruption',
         'health',
         'gender inequality',
         'rape',
         'ebola',
         'aids',
         'hiv',
         'technology',
         'artificial intelligence',
         'military',
         'war',
         'climate change',
         'starvation',
         'food shortage',
         'dehydration',
         'water shortage',
         'attack',
         'aggression',
         'logistics',
         'nutrition',
         'protection',
         'shelter',
         'drinking water',
         'sanitation',
         'hygiene',
         'refugee camp',
         'education',
         'emergency communication system',
         'food security',
         'human rights',
         'children',
         'pregnancy',
         'old age',
         'justice',
         'law',
         'maintenance (technical)',
         'homelessness',
         'art',
         'culture',
         'indigenous peoples'
       ]

api_key = '34e73062364d41b180a6f6a9625bff79'

In [3]:
data_location = '/Users/thiesgehrmann/Downloads/TWB/documents/hackathon-for-good-2019_TWB-challenge_files/*.sdlxliff'
xliff_files = glob.glob(data_location)
xliff = [ TWB.XLIFF(f) for f in xliff_files[:100]]

## Load the API

In [4]:
api = TWB.API(tags, api_key)

## Get data for xliffs from the API

In [5]:
api.get(xliff)

[{'target_lsc': code(language=language(iso3='BEN', iso2='BN', english='BENGALI', french='BENGALI', german='BENGALI'), script=script(code='LATN', number=215, name='LATIN', alias='LATIN', direction='L-TO-R', version=1.0, characters=1366.0, remarks='SEE LATIN SCRIPT IN UNICODE'), country=country(country='INDIA', iso2='IN', iso3='IND', un=356)),
  'topics': {'children': {'distance': 2.0,
    'relevant_country_languages': {'JPN': {'frequency': 6,
      'language_relevant': False},
     'OMN': {'frequency': 3, 'language_relevant': False},
     'GTM': {'frequency': 2, 'language_relevant': False},
     'PAK': {'frequency': 1, 'language_relevant': False},
     'MLI': {'frequency': 1, 'language_relevant': False},
     'LBN': {'frequency': 1, 'language_relevant': False},
     'SAU': {'frequency': 1, 'language_relevant': False},
     'YEM': {'frequency': 1, 'language_relevant': False},
     'ETH': {'frequency': 1, 'language_relevant': False},
     'LBY': {'frequency': 1, 'language_relevant': False